# Parse Race Results

For this project, I have to fetch a ton of marathon results. [Marathon Guide](http://www.marathonguide.com/index.cfm) has database with a ton of race results, but it isn't straightforward to programmatically extract that data. Each race has a unique ID, and each yearly event of that race has its own unique ID as well. Once you get to the page of an actual event, say the 2023 Boston Marathon, the race results live on a table on that page. So we'll have to programmatically do the following:

1. search for a race
2. find that race's ID
3. visit the page for that race
4. find the IDs for each year of that race
5. for a particular year, scrape the data.

## Import Modules

Let's import what we need to parse the Marathon Guide site results.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

## Run A Query for the Boston Marathon

Now let's search for "Boston" to see if we can find the Boston Marathon.

In [2]:
url = "http://www.marathonguide.com/races/search.cfm"
searchParams = {
    "MarName":"Boston",
    "state":0,
    "Country":0,
    "BeginDate":"",
    "EndDate":"",
    "submit":"Search"
}
res = requests.post(url,data=searchParams)

## Parse Marathon Search with BeautifulSoup

We can read in the results content with BeautifulSoup. The search results are all linked to their page, referenced by an ID labelled as "MIDD". We can use this link to peel off the MIDD for each race in the search result via `MIDD\s*=\s*([\S\s]+)`, where

* `MIDD` checks for the text "MIDD"
* `\s*` selects all whitespace if there is any
* `=` Will match the "=" sign
* `([\S\s]+)` select all characters.

In [3]:
# Read in the results of the query with BeautifulSoup
soup = BeautifulSoup(res.content)

# Loop through all <a> tags and find those with "Boston" in the name
# TODO: Ignore any <strike>'ed results
for a in soup.find_all("a"):
    if a.text == "Boston Marathon":
        link = a.get_attribute_list('href')
        midd = re.search('MIDD\s*=\s*([\S\s]+)', link[0]).group(1)
        race = a.text
        print(f"Race: {race}, MIDD: {midd}, Link: {link}")

Race: Boston Marathon, MIDD: 15240415, Link: ['../races/racedetails.cfm?MIDD=15240415']


## Parse the Events for the Race

Now that we've searched for a race, we need to select the event for a specific year. This is on the race details page, we just need to parse it out.

In [4]:
url = 'http://www.marathonguide.com/races/racedetails.cfm'
params = {
    "MIDD": midd
}
res = requests.get(url,params)

years = [str(i) for i in range(2000,2024)]
soup = BeautifulSoup(res.content)
dfList = []
for a in soup.find_all("a"):
    if a.text in years:
        link = a.get_attribute_list('href')
        midd = re.search('MIDD\s*=\s*([\S\s]+)', link[0]).group(1)
        year = a.text
        dfList.append(pd.DataFrame([midd,year]))

df = pd.concat(dfList,axis=1).T
df.columns = ("midd","year")
df = df.sort_values("year").reset_index().drop('index',axis=1)
df.tail()

,midd,year
18,15180416,2018
19,15190415,2019
20,15211011,2021
21,15220418,2022
22,15230417,2023


## Get the Page for Results of the Most Recent Marathon

Now we get the page for a particular event; the most recent Boston Marathon. We'll parse some initial results from the page, such as the number of finishers, and the number of finishers registered as male and female.

In [5]:
# Just get the last year's results
year = df["year"].to_list()[-1]
midd = df["midd"].to_list()[-1]

# Make the request for the event results
url = f"http://www.marathonguide.com/results/browse.cfm"
params = {
    "MIDD": midd
}
res = requests.get(url,params)

# Parse out the number of finishers, the number of male finishers, and the number of female finishers
nFinishers = int(res.text.split("Finishers: ")[1].split(",")[0])
nMales = int(res.text.split("Males - ")[1].split(",")[0])
nFemales = int(res.text.split("Females - ")[1].split("\n")[0])
print(f"Finishers: {nFinishers}, Male Finishers: {nMales}, Female Finishers: {nFemales}")

Finishers: 26600, Male Finishers: 15171, Female Finishers: 11405


## Parse Event Results

The page requires users to submit a form to view the data, so let's now scrape that data for an event. The form breaks down the search in 100 results at a time, so let's start with that.

In [6]:
# Set search parameters
gender = "B"
start = 1
end = 100

# Set the form URL
url = 'http://www.marathonguide.com/results/makelinks.cfm'

# Set the data to POST
data = {
  'RaceRange': f'{gender},{start},{end},{nFinishers}', 
  'RaceRange_Required': 'You must make a selection before viewing results.', 
  'MIDD': f'{midd}', 
  'SubmitButton': 'View'}

# Set the headers for the POST
headers = {
    "Referer": f"http://www.marathonguide.com/results/browse.cfm?MIDD={midd}",
    "User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.116 Safari/537.36"
}

# Submit the request
results = requests.post(url, data=data, headers=headers)

# Parse the content
soup = BeautifulSoup(results.content)

# Get the column headers from the table
colNames = [th.get_text().strip() for th in soup.find_all("th")]

# Create empty list to append
data = []

# Get the data from the table rows
rows = soup.find_all("tr")

# Loop through table rows
for row in rows:
    
    # Create empty list to append
    rowData = []

    # Get all of the cells in the row
    cells = row.find_all("td")

    # Find the rows with the right number of columns
    if len(cells) == len(colNames):

        # Loop through all of the table cells in the row
        for cell in cells:

            # Append the cell text to the row data
            rowData.append(cell.get_text().strip())    

    # Append the row data to the data list
    data.append(rowData)

dfResults = pd.DataFrame(data,columns=colNames)
dfResults = dfResults.dropna().reset_index().drop('index',axis=1).copy()
dfResults.head()

,"Last Name, First Name(Sex/Age)",Time,OverAllPlace,Sex Place/Div Place,DIV,"City, State, Country",AG Time*,BQ*
0,Evans Chebet (M34),2:05:54,1,1 / 1,M0-39,"Kapsabet, Kenya",2:05:54,BQ
1,Gabriel Geay (M26),2:06:04,2,2 / 2,M0-39,"Tampa, FL, Tanzania",2:06:04,BQ
2,Benson Kipruto (M32),2:06:06,3,3 / 3,M0-39,"Kapsabet, Kenya",2:06:06,BQ
3,Albert Korir (M29),2:08:01,4,4 / 4,M0-39,Kenya,2:08:01,BQ
4,Zouhair Talbi (M28),2:08:35,5,5 / 5,M0-39,Morocco,2:08:35,BQ
